In [1]:
import tensorflow as tf
import pickle
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def load_preprocessed_data(name):
    with open(name,'rb') as f:
        data = pickle.load(f)
    return data

In [3]:
train = load_preprocessed_data('processed_data/train.pickle')
test = load_preprocessed_data('processed_data/test.pickle')
val = load_preprocessed_data('processed_data/val.pickle')
input_shape = load_preprocessed_data('processed_data/input_shape.pickle')

In [4]:
class DataSequence(tf.keras.utils.Sequence):
    def __init__(self,data,batch_size=32):
        self.data = data
        self.batch_size = batch_size
    
    def __len__(self):
        return int(np.ceil(len(self.data) / self.batch_size))

    def __getitem__(self,idx):
        batch = self.data[idx * self.batch_size : (idx + 1) * self.batch_size]
        return self.data_gen(batch)

    def data_gen(self,data):
        images,labels = list(),list()
        for img,label in data:
            try: 
                images.append(img)
                labels.append(label)
            except: continue
        return np.array(images),np.array(labels)


In [5]:
train_data = DataSequence(train,4)
test_data = DataSequence(test,1)
val_data = DataSequence(val,1)

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape, name='Input_Layer'),

    tf.keras.layers.Conv2D(64,kernel_size=(3,3),name='Conv_Layer_1'),
    tf.keras.layers.Activation('relu',name='Conv_Act_1'),
    tf.keras.layers.MaxPool2D((3,3),name='Conv_MaxPool_1'),
    
    tf.keras.layers.Conv2D(128,kernel_size=(3,3),name='Conv_Layer_2'),
    tf.keras.layers.Activation('relu',name='Conv_Act_2'),
    tf.keras.layers.MaxPool2D((3,3),name='Conv_MaxPool_2'),

    tf.keras.layers.Conv2D(256,kernel_size=(3,3),name='Conv_Layer_3'),
    tf.keras.layers.Activation('relu',name='Conv_Act_3'),
    tf.keras.layers.MaxPool2D((3,3),name='Conv_MaxPool_3'),

    tf.keras.layers.Flatten(name='Flatten'),
    
    tf.keras.layers.Dense(512,name='Dense_Layer_1'),
    tf.keras.layers.Activation('relu',name='Dense_Act_1'),

    tf.keras.layers.Dropout(0.3,name='Dropout_layer_1'),

    tf.keras.layers.Dense(256,name='Dense_Layer_2'),
    tf.keras.layers.Activation('relu',name='Dense_Act_2'),

    tf.keras.layers.Dense(128,name='Dense_Layer_3'),
    tf.keras.layers.Activation('relu',name='Dense_Act_3'),

    tf.keras.layers.Dense(1,name='Output_Layer'),
    tf.keras.layers.Activation('sigmoid',name='Output_Activation'),
])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = ['accuracy']
)

In [10]:
with tf.device('/cpu'):
    history = model.fit(train_data,epochs=10,verbose=1,validation_data=val_data)

Epoch 1/10
1309/1309 [==============================] - 1318s 1s/step - loss: 0.2509 - accuracy: 0.9039 - val_loss: 0.9105 - val_accuracy: 0.6875
Epoch 2/10
 891/1309 [===================>..........] - ETA: 6:52 - loss: 0.1225 - accuracy: 0.9579

KeyboardInterrupt: 

In [ ]:
test_data = [(i[0],i[1]) for i in test_data if i[1].size]

In [ ]:
y_pred = list()
y_test = list()
for i,j in test_data:
    try:
        p = model.predict(i)
        y_pred.append(1 if p[0][0]>0.5 else 0)
        y_test.append(j)
    except: continue

In [ ]:
conf_mat = tf.math.confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
sns.heatmap(conf_mat,annot=True)
plt.show()

In [ ]:
model.save('classifier.h5')